In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from config import path_to_repository
from analysis import BallBearingData # class for extracting all data from application and reapplication for ADC, IMU, and Camera Triggers
from analysis import DLC3DBendAngles # class for taking DLC 3d point data and converting to angles 
from analysis import bender_class # class for normalizing adc data, analyzing autobender tests
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Upload ball bearing and beaker Trials from particpant
# Upload only the parent *date* folder (e.g., "9_19_25").
# If something fails, verify your folder structure matches the "9_19_25" example.

root_dir="CSV Data/11_2_25"
folder_suffix="imu_slow"

#10_28_25 (Tom)
#10_29_25 (bending data might be okay...check)
#11_2_25 (good)
#11_4_25 (good)

##################### raw adc or imu data for any test ###############################

bb = BallBearingData(
    root_dir=root_dir, # all that is needed is to upload the parent date folder.  If there are issues with this, check the structure of folder 9_19_25 as an example 
    path_to_repo=path_to_repository,
    n_trials_per_set=4,   # 15 trials of ball bearing reach to pinch, 15 trials of beaker reach to grasp, and the same for reapplication
    files_per_trial=7,
    folder_suffix=folder_suffix,     # this suffic contains data from ball bearing task  
    # Valid folder_suffix values (examples): check the 9_19_25 folder for CSV naming patterns/suffixes
)
df_first  = bb.load_first() # first application
df_second = bb.load_second() # reapplication

Found 4 *_imu_slow folders total (case-insensitive).
  example: C:\Users\toppe\OneDrive - CSU Maritime Academy\Documents\GitHub\Strain-Sensor-\CSV Data\11_2_25\2025_11_02_14_39_02_imu_slow
First set range: 2025_11_02_14_39_02_imu_slow → 2025_11_02_14_39_53_imu_slow
Second set range: 2025_11_02_15_42_55_imu_slow → 2025_11_02_15_43_23_imu_slow
[WARN] ball_bearing_first: Some trials do not have exactly 7 CSVs:
  • Trial 01: 2025_11_02_14_39_02_imu_slow has 6 CSVs (including nested)
  • Trial 02: 2025_11_02_14_39_53_imu_slow has 6 CSVs (including nested)
First set range: 2025_11_02_14_39_02_imu_slow → 2025_11_02_14_39_53_imu_slow
Second set range: 2025_11_02_15_42_55_imu_slow → 2025_11_02_15_43_23_imu_slow
[WARN] ball_bearing_second: Some trials do not have exactly 7 CSVs:
  • Trial 01: 2025_11_02_15_42_55_imu_slow has 6 CSVs (including nested)
  • Trial 02: 2025_11_02_15_43_23_imu_slow has 6 CSVs (including nested)


# Get per-trial camera timestamp tables from flir.mat (prefix 'ts*') and DLC 3D point data (two separate dataframes)

This is where you should probably also load the DLC data and combine with cam_trials_first and cam_trials_second.    See my BallBearingClass for examples of how to do that.  

In [8]:
# FIRST application (camera + DLC should be 'first_cam')
cam_trials_first = bb.extract_mat_dfs_by_trial(
    df_first, mat_name="flir.mat", prefix="ts",
    add_labels=True, trial_base=1, set_label="first_cam", include_path=True
)
dlc3d_trials_first = bb.extract_dlc3d_dfs_by_trial(
    df_first,
    add_labels=True, trial_base=1, set_label="first_cam", include_path=True
)

# SECOND application (camera + DLC should be 'second_cam')
cam_trials_second = bb.extract_mat_dfs_by_trial(
    df_second, mat_name="flir.mat", prefix="ts",
    add_labels=True, trial_base=1, set_label="second_cam", include_path=True
)
dlc3d_trials_second = bb.extract_dlc3d_dfs_by_trial(
    df_second,
    add_labels=True, trial_base=1, set_label="second_cam", include_path=True
)

# --- Build DLC angle tables for FIRST and SECOND sets ---


# 2) Coerce headers to the expected MultiIndex format
dlc3d_trials_first  = [bb._coerce_dlc3d_multiindex(d)  for d in dlc3d_trials_first]
dlc3d_trials_second = [bb._coerce_dlc3d_multiindex(d) for d in dlc3d_trials_second]

# 3) Compute DLC-derived angles (returns per-trial augmented dfs + tall table)
aug_first,  tall_first  = bb.compute_dlc3d_angles_by_trial(
    dlc3d_trials_first,  set_label="first_cam",  signed_in_plane=True
)
aug_second, tall_second = bb.compute_dlc3d_angles_by_trial(
    dlc3d_trials_second, set_label="second_cam", signed_in_plane=True
)

# (Optional sanity)
print("len(aug_first) vs cam_trials_first:",  len(aug_first),  "vs", len(cam_trials_first))
print("len(aug_second) vs cam_trials_second:", len(aug_second), "vs", len(cam_trials_second))




len(aug_first) vs cam_trials_first: 2 vs 2
len(aug_second) vs cam_trials_second: 2 vs 2


# Convert 3d Data from DLC to angles and append that column to dataframes created in previous cell

See camera_encoder_cal notebook for how this is done using a class method.  

Once you have the angles, can match timestamps below and do error analysis

In [11]:
# ==== prerequisites ====
import pandas as pd

ALLOWED = {"first_block","first_cam","second_block","second_cam"}
REMAP = {
    "first cam":"first_cam","first block":"first_block",
    "second cam":"second_cam","second block":"second_block",
}

def _canon_label(s: str) -> str:
    s = (s or "").strip()
    s = REMAP.get(s, s)
    if s not in ALLOWED:
        raise ValueError(f"Unexpected set_label='{s}'. Must be one of {sorted(ALLOWED)}")
    return s

def _strip_dlc_cols(list_of_dfs):
    """Remove any previously attached DLC metric columns (ending with '_dlc')."""
    out = []
    for df in (list_of_dfs or []):
        if df is None or df.empty:
            out.append(df); continue
        drop_cols = [c for c in df.columns if str(c).endswith("_dlc")]
        out.append(df.drop(columns=drop_cols, errors="ignore"))
    return out

def labels_from_cam(cam_list):
    """Robust per-trial label extraction from camera(+dlc) dfs."""
    out = []
    for i, df in enumerate(cam_list):
        if df is None or df.empty:
            continue
        if "trial" in df.columns:
            s = pd.to_numeric(df["trial"], errors="coerce").dropna().astype(int)
            if not s.empty:
                out.append(int(s.mode().iat[0])); continue
        out.append(len(out)+1)  # 1-based fallback
    return out

def duplicate_trial_labels(labels):
    """[1,2,3] -> [1,1,2,2,3,3]"""
    return [x for t in labels for x in (t, t)]

def duplicate_set_labels_for_four(cal_cam_list, cam_label: str, block_label: str, trial_labels=None):
    """
    Duplicate each per-trial camera(+dlc) df so we get *both* entries:
    cam_label (e.g., 'first_cam') and block_label (e.g., 'first_block').
    Also stamps a consistent 'trial' on both copies.
    Order: [cam, block, cam, block, ...]
    """
    cam_label   = _canon_label(cam_label)
    block_label = _canon_label(block_label)

    out = []
    for i, df in enumerate(cal_cam_list):
        if df is None or df.empty:
            out.extend([df, df]); continue

        # decide the trial id for this element
        if trial_labels is not None:
            t = int(trial_labels[i])
        elif "trial" in df.columns:
            t = int(pd.to_numeric(df["trial"], errors="coerce").dropna().mode().iat[0])
        else:
            t = i+1  # 1-based fallback

        a = df.copy(); a["set_label"] = cam_label;   a["trial"] = t
        b = df.copy(); b["set_label"] = block_label; b["trial"] = t
        out.extend([a, b])
    return out

def show_labels(lst, name):
    s=set()
    for d in lst or []:
        if d is not None and not d.empty and "set_label" in d.columns:
            s |= set(d["set_label"].astype(str).unique())
    print(name, "->", sorted(s))


# ==== 2) attach DLC angles into NEW variables (avoid overwriting raw lists) ====
cam_trials_first_cam = bb.attach_dlc_angles_to_cam_by_trial(
    cam_trials_first,  aug_first,
    add_labels=True, set_label="first_cam", trial_base=1
)
cam_trials_second_cam = bb.attach_dlc_angles_to_cam_by_trial(
    cam_trials_second, aug_second,
    add_labels=True, set_label="second_cam", trial_base=1
)


# Trial labels derived from the stamped 'trial' columns in the *_cam lists
labels_first  = labels_from_cam(cam_trials_first_cam)
labels_second = labels_from_cam(cam_trials_second_cam)

# ==== 3) duplicate to get the FOUR canonical set_labels ====
cam_plus_dlc_first_four = duplicate_set_labels_for_four(
    cam_trials_first_cam, cam_label="first_cam", block_label="first_block",
    #trial_labels=labels_first
)
cam_plus_dlc_second_four = duplicate_set_labels_for_four(
    cam_trials_second_cam, cam_label="second_cam", block_label="second_block",
    #trial_labels=labels_second
)

labels_first_four  = duplicate_trial_labels(labels_first)
labels_second_four = duplicate_trial_labels(labels_second)

# ==== sanity ====
show_labels(cam_plus_dlc_first_four,  "first_four")    # -> ['first_block','first_cam']
show_labels(cam_plus_dlc_second_four, "second_four")   # -> ['second_block','second_cam']




first_four -> ['first_block', 'first_cam']
second_four -> ['second_block', 'second_cam']


## IMU Analysis of Wrist Bend Angles Here

In [15]:

# 2) Extract IMU trials (same calls as before)
imu_trials_first  = bb.extract_imu_dfs_by_trial(df_first)      # list length = bb.n_trials_per_set
imu_trials_second = bb.extract_imu_dfs_by_trial(df_second)

print(f"First set:  {len(imu_trials_first)} IMU trials")
print(f"Second set: {len(imu_trials_second)} IMU trials")


# 3) Access a single trial (unchanged)
trial1_imu = imu_trials_first[0]   # 0-based index -> trial 1




First set:  2 IMU trials
Second set: 2 IMU trials


## Compute bend angle from two IMU's

In [18]:
# ========================= IMU → joint-angle math overview =========================
# We mount two IMUs:
#   • IMU1 ("fixed"): reference segment (watch)
#   • IMU2 ("moving"): distal segment (hand/palm)
#
# Each IMU streams orientation as a unit quaternion q = [w, x, y, z] in some order
# (here 'wxyz'; change QUAT_ORDER if your data is 'xyzw'). A unit quaternion
# represents a 3D rotation R(q) with no gimbal lock and stable interpolation.
#
# Goal: compute the bend angle between a chosen axis on the fixed segment and a
# chosen axis on the moving segment, *as seen in the fixed segment’s frame*.
#
# Math (conceptual):
#   1) Convert both quaternions to rotation matrices:
#        R_fixed = R(q_fixed), R_moving = R(q_moving)
#
#   2) Bring the moving frame into the fixed frame via the relative rotation:
#        R_rel = R_fixed^T · R_moving
#
#   3) Choose body axes (e.g., axis pointing down the index-finger direction):
#        e_fix  = unit axis on the fixed body (x/y/z in IMU1's body frame)
#        e_move = unit axis on the moving body (x/y/z in IMU2's body frame)
#        The strings FIXED_AXIS and MOVING_AXIS select which canonical axis ('x', 'y', or 'z').
#
# Notes:
# - Pick axes that align with the physical direction you want to measure (e.g., IMU1's y-axis along forearm, IMU2's x-axis along palm/fingers).
# - If your hardware mounting changes, update FIXED_AXIS/MOVING_AXIS accordingly.

# Examples (choose letters that match how your sensors are physically mounted):
#
# A) Common mounting:
#    - IMU1 (forearm/watch) mounted so its body y-axis points along the forearm (elbow→wrist).
#    - IMU2 (palm) mounted so its body x-axis points down the fingers (wrist→index tip).
#    -> FIXED_AXIS  = "y"   # e_fix = [0,1,0] in IMU1 body frame
#       MOVING_AXIS = "x"   # e_move = [1,0,0] in IMU2 body frame
#
# B) Rolled/yawed mount where IMU1's body z-axis ends up pointing down the index finger:
#    - After mounting, the direction you care about corresponds to IMU1's z-axis.
#    -> FIXED_AXIS  = "z"
#       MOVING_AXIS = "x"   # (or "y"/"z") pick the IMU2 axis that tracks the palm/fingers best
#
# C) IMU2 flipped 180 deg around its y-axis (so x flips sign):
#    - If your angles look inverted, try switching MOVING_AXIS letter:
#    -> previous: MOVING_AXIS = "x"
#       try:      MOVING_AXIS = "z"  (or "y") depending on which axis now aligns with finger direction
#
# D) Both sensors mounted with their x-axes along the limb segments:
#    -> FIXED_AXIS  = "x"
#       MOVING_AXIS = "x"
#
# E) Quick calibration tip:
#    - Hold a neutral pose, compute R_fixed and R_moving from quats.
#    - Test which body axis (x/y/z) for each IMU has the largest dot product with your intended anatomical direction.
#      Pick that letter as FIXED_AXIS / MOVING_AXIS.
#
#   4) Rotate the moving axis into the fixed frame:
#        v = R_rel · e_move
#
#   5) Bend angle is the angle between v and e_fix in the fixed frame:
#        theta = arccos( clamp( v · e_fix, -1, 1 ) )
#
#      • If you need a *signed* angle in a specific plane, you can project onto a plane
#        and use cross products to set the sign. The pipeline here computes a principal
#        (unsigned) angle consistent with the axis choices.
#      • Units: theta is computed in radians then converted to degrees for convenience.
#
# Why axis choices matter:
#   • Your physical mounting determines which body axis of each IMU best tracks the
#     bending direction. For Tom’s data, IMU1 (watch) was against a vertical wall and
#     IMU2 (palm) faced along the hand toward the fingers, so FIXED_AXIS and MOVING_AXIS
#     are set to align with those physical directions. If your mounting changes, update
#     FIXED_AXIS/MOVING_AXIS to match the true “bend” axes.
#
# Quaternion order:
#   • Many devices output [x, y, z, w] (xyzw) while math libraries prefer [w, x, y, z] (wxyz).
#     QUAT_ORDER tells the code how to read your columns and map them correctly before
#     building R(q). If this is wrong, the computed angle will be nonsense.
#



# ---- IMU processing params  ----
TRIAL_LEN_SEC = 10.0                 # trial duration (s) used to synthesize time_s if needed
QUAT_COLS     = ("euler1", "euler2")   # names of the two quaternion columns/blocks (IMU1, IMU2)
                                      # (or use ("euler1","euler2") if your pipeline expects eulers there)

# For Tom's setup:
#   - IMU1 (watch) placed against a vertical wall (reference segment)
#   - IMU2 (palm) facing down the hand axis toward the fingers (moving segment)
# The chosen axes below reflect those mountings. Adjust if your hardware orientation changes.
FIXED_AXIS    = "x"                  # body axis of IMU1 used as the reference direction (e_fix)
MOVING_AXIS   = "x"                  # body axis of IMU2 whose motion defines the bend (e_move)
QUAT_ORDER    = "wxyz"               # order of quaternion components in your columns ('wxyz' or 'xyzw')

# 3) Compute joint angle per trial, in place (adds: time_s, imu_joint_deg_rx_py)
bb.imu_augment_trials_inplace(
    aligned_first,            # kept for API symmetry; not used internally
    imu_trials_first,
    trial_len_sec=TRIAL_LEN_SEC,
    quat_cols=QUAT_COLS,
    fixed_axis=FIXED_AXIS,
    moving_axis=MOVING_AXIS,
    quat_order=QUAT_ORDER,
)
bb.imu_augment_trials_inplace(
    aligned_second,
    imu_trials_second,
    trial_len_sec=TRIAL_LEN_SEC,
    quat_cols=QUAT_COLS,
    fixed_axis=FIXED_AXIS,
    moving_axis=MOVING_AXIS,
    quat_order=QUAT_ORDER,
)

# 4) Build tall (long-form) frames for plotting/analysis.
#    imu_collect_tall(...) re-runs the same geometry on the provided trials and returns a
#    concatenated dataframe with columns like:
#      ['set_label','trial','time_s','timestamp','imu_joint_deg_rx_py', ...]
A_all = bb.imu_collect_tall(
    imu_trials_first, "first",
    trial_len_sec=TRIAL_LEN_SEC,
    quat_cols=QUAT_COLS,
    fixed_axis=FIXED_AXIS,
    moving_axis=MOVING_AXIS,
    quat_order=QUAT_ORDER,
)
B_all = bb.imu_collect_tall(
    imu_trials_second, "second",
    trial_len_sec=TRIAL_LEN_SEC,
    quat_cols=QUAT_COLS,
    fixed_axis=FIXED_AXIS,
    moving_axis=MOVING_AXIS,
    quat_order=QUAT_ORDER,
)

# After imu_collect_tall:
for tag, T in [("A_all", A_all), ("B_all", B_all)]:
    if T is None or T.empty:
        print(f"[{tag}] is EMPTY"); continue
    if "trial" not in T.columns:
        # fallback: assume trials are contiguous chunks; or stamp 1 if unknown
        T["trial"] = 1
    if "set_label" not in T.columns:
        T["set_label"] = "first_cam" if tag=="A_all" else "second_cam"
    print(f"[{tag}] trials:", sorted(pd.to_numeric(T["trial"], errors="coerce").dropna().astype(int).unique()))



print(f"[A] non-empty trials: {A_all['trial'].nunique() if not A_all.empty else 0}, rows: {len(A_all)}")
print(f"[B] non-empty trials: {B_all['trial'].nunique() if not B_all.empty else 0}, rows: {len(B_all)}")

# 5) Quick plot
fig, axes = plt.subplots(2, 1, figsize=(10, 7), sharex=True, sharey=True)

for tr in (sorted(A_all["trial"].unique()) if not A_all.empty else []):
    d = A_all[A_all["trial"] == tr]
    axes[0].plot(d["time_s"], d["imu_joint_deg_rx_py"], ".", markersize=2, color="blue")
axes[0].set_title("Set A: θ_IMU(t)")
axes[0].set_ylabel("θ_IMU (deg)")
axes[0].grid(alpha=0.25)

for tr in (sorted(B_all["trial"].unique()) if not B_all.empty else []):
    d = B_all[B_all["trial"] == tr]
    axes[1].plot(d["time_s"], d["imu_joint_deg_rx_py"], ".", markersize=2, color="red")
axes[1].set_title("Set B: θ_IMU(t)")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("θ_IMU (deg)")
axes[1].grid(alpha=0.25)

plt.tight_layout()
plt.show()



NameError: name 'aligned_first' is not defined

In [20]:
# Match trial imu timestamps to camera trigger timestamps -> need for error analysis
# See jupyter notebook camera_encoder_cal.ipnyb.  This file compares the camera 3d data with timestamps to encoder data from autobender with timestamps

cam_plus_dlc_first_four

aligned_first_imu = bb.align_theta_all_to_cam_for_set(
    A_all, cam_plus_dlc_first_four,
    enc_time_col="timestamp",                # will fall back to 'time'/'t_sec' if needed
    cam_time_col="ts_25185174",              # <— pick the correct one
    tolerance=1000000000000000,                       # start generous
    theta_col="imu_joint_deg_rx_py",
    drop_unmatched=True,
)

aligned_second_imu = bb.align_theta_all_to_cam_for_set(
    B_all, cam_plus_dlc_second_four,
    enc_time_col="timestamp",                # will fall back to 'time'/'t_sec' if needed
    cam_time_col="ts_25185174",              # <— pick the correct one
    tolerance=1000000000000000,                       # start generous
    theta_col="imu_joint_deg_rx_py",
    drop_unmatched=True,
)

aligned_first_imu[0]

# Keep only rows where DLC bend angle ≤ 90° (and drop NaNs)
# ---- Keep only rows with DLC and IMU angles within [0, 90] deg ----
dlc_col = "metric_wrist_bend_deg_deg_dlc"   # change if your DLC column name differs
imu_col = "imu_joint_deg_rx_py"             # IMU angle column from your alignment
deg_lo, deg_hi = 0.0, 90.0

def filter_angles_in_range(aligned_list, dlc_col, imu_col, lo=0.0, hi=90.0, dropna=True):
    """
    For each aligned trial DataFrame in `aligned_list`, keep only rows where:
      - (if present) DLC angle ∈ [lo, hi]
      - (if present) IMU angle ∈ [lo, hi]
    If a column is missing, it won't be used in the filter.
    """
    out = []
    for df in aligned_list:
        if df is None or df.empty:
            out.append(df); continue

        mask = pd.Series(True, index=df.index)

        if dlc_col in df.columns:
            a = pd.to_numeric(df[dlc_col], errors="coerce")
            m = (a >= lo) & (a <= hi)
            if dropna: m &= a.notna()
            mask &= m

        if imu_col in df.columns:
            b = pd.to_numeric(df[imu_col], errors="coerce")
            m = (b >= lo) & (b <= hi)
            if dropna: m &= b.notna()
            mask &= m

        out.append(df.loc[mask].reset_index(drop=True))
    return out

aligned_first_imu  = filter_angles_in_range(aligned_first_imu,  dlc_col, imu_col, lo=deg_lo, hi=deg_hi)
aligned_second_imu = filter_angles_in_range(aligned_second_imu, dlc_col, imu_col, lo=deg_lo, hi=deg_hi)

cam_plus_dlc_first_four



NameError: name 'A_all' is not defined

In [22]:
# Plot IMU wrist angle from DLC angle

# Pick a trial (trial 3 => index 2)
t3A = aligned_first_imu[1].reset_index(drop=True)
t3B = aligned_second_imu[1].reset_index(drop=True)

def plot_two_vs_index(df, title):
    col_dlc = "metric_wrist_bend_deg_deg_dlc"
    col_imu = "imu_joint_deg_rx_py"
    fig, ax = plt.subplots(figsize=(10, 4))

    if col_dlc in df.columns:
        ax.plot(df.index, df[col_dlc], ".", markersize=2, label="wrist_bend (DLC)")
    else:
        print(f"[warn] '{col_dlc}' not found. Available: {list(df.columns)}")

    if col_imu in df.columns:
        ax.plot(df.index, df[col_imu], ".", markersize=2, label="imu_joint (IMU)")
    else:
        print(f"[warn] '{col_imu}' not found. Available: {list(df.columns)}")

    ax.set_title(title)
    ax.set_xlabel("row index")
    ax.set_ylabel("angle (deg)")
    ax.grid(alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.show()

plot_two_vs_index(t3A, "Set A — Trial 3: DLC wrist bend vs IMU joint angle (index)")
plot_two_vs_index(t3B, "Set B — Trial 3: DLC wrist bend vs IMU joint angle (index)")


NameError: name 'aligned_first_imu' is not defined

In [24]:
res = bb.plot_imu_vs_dlc_twopanel(
    aligned_first_imu=aligned_first_imu,
    aligned_second_imu=aligned_second_imu,
    dlc_angle_col="metric_wrist_bend_deg_deg_dlc",
    imu_angle_col="imu_joint_deg_rx_py",
    trials_first=None,
    trials_second=None,
    max_abs_dt_ms=100.0,
    verbose=True,
)


NameError: name 'aligned_first_imu' is not defined

In [26]:
res_imu = bb.plot_imu_dlc_abs_error_boxplots(
    aligned_first_imu=aligned_first_imu,
    aligned_second_imu=aligned_second_imu,
    dlc_angle_col="metric_wrist_bend_deg_deg_dlc",
    imu_angle_col="imu_joint_deg_rx_py",
    # choose specific trials (examples):
    # trials_first=[1, 3], trials_second=[1, 2],   # accepts 1- or 0-based
    trials_first=None,   # auto-pick first two non-empty
    trials_second=None,  # auto-pick first two non-empty
    max_abs_dt_ms=100.0,
    verbose=True,
)
print(res_imu["stats"])


NameError: name 'aligned_first_imu' is not defined